In [2]:
import pandas as pd
import numpy as np
import time
import nba_api

import json

from nba_api.stats.static import players
from nba_api.stats.static import teams

from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import teamyearbyyearstats
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
##from fastparquet import ParquetFile
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import leaguedashplayershotlocations
from nba_api.stats.endpoints import leaguedashplayerptshot

from nba_api.stats.endpoints import commonplayerinfo

#Add new data by season to dataframe



In [3]:
#Player advanced stats
player_season_stats = leaguedashplayerstats.LeagueDashPlayerStats(measure_type_detailed_defense = 'Advanced', 
                                        season_type_all_star = 'Regular Season', season = '2021-22').get_data_frames()[0]

In [4]:
#Getting player shot % stats
player_shot_stats = leaguedashplayerptshot.LeagueDashPlayerPtShot( 
                                        season_type_all_star = 'Regular Season', season = '2010-11').get_data_frames()[0]

In [293]:
#Manipulating player shot % data
player_shot_stats.columns = player_shot_stats.columns.str.lower()
player_shot_stats = player_shot_stats.drop(columns = ['player_last_team_id', 'player_name', 'player_last_team_abbreviation', 'gp', 'g', 'fga_frequency', 'fgm', 'fga', 'fg_pct'], axis = 1)

In [294]:
test = pd.merge(player_season_stats,player_shot_stats, left_on = 'PLAYER_ID', right_on = 'player_id', how = 'outer')

In [295]:
print(len(test))
print(len(player_shot_stats))
print(len(player_season_stats))

605
596
605


In [296]:
test

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,age,efg_pct,fg2a_frequency,fg2m,fg2a,fg2_pct,fg3a_frequency,fg3m,fg3a,fg3_pct
0,203932,Aaron Gordon,Aaron,1610612743,DEN,26.0,75,46,29,0.613,...,26.0,0.573,0.688,347.0,574.0,0.605,0.312,87.0,260.0,0.335
1,1630565,Aaron Henry,Aaron,1610612755,PHI,22.0,6,6,0,1.000,...,22.0,0.200,0.800,1.0,4.0,0.250,0.200,0.0,1.0,0.000
2,1628988,Aaron Holiday,Aaron,1610612756,PHX,25.0,63,34,29,0.540,...,25.0,0.504,0.695,112.0,235.0,0.477,0.305,39.0,103.0,0.379
3,1630174,Aaron Nesmith,Aaron,1610612738,BOS,22.0,52,32,20,0.615,...,22.0,0.481,0.368,41.0,67.0,0.612,0.632,31.0,115.0,0.270
4,1630598,Aaron Wiggins,Aaron,1610612760,OKC,23.0,50,13,37,0.260,...,23.0,0.525,0.591,114.0,199.0,0.573,0.409,42.0,138.0,0.304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,203897,Zach LaVine,Zach,1610612741,CHI,27.0,67,37,30,0.552,...,27.0,0.555,0.599,380.0,710.0,0.535,0.401,185.0,475.0,0.389
601,1630285,Zavier Simpson,Zavier,1610612760,OKC,25.0,4,1,3,0.250,...,25.0,0.375,0.846,18.0,44.0,0.409,0.154,1.0,8.0,0.125
602,1630192,Zeke Nnaji,Zeke,1610612743,DEN,21.0,41,24,17,0.585,...,21.0,0.616,0.570,59.0,106.0,0.557,0.430,37.0,80.0,0.463
603,1630533,Ziaire Williams,Ziaire,1610612763,MEM,20.0,62,41,21,0.661,...,20.0,0.536,0.428,112.0,178.0,0.629,0.572,74.0,238.0,0.311


In [297]:
player_shot_stats.columns

Index(['player_id', 'age', 'efg_pct', 'fg2a_frequency', 'fg2m', 'fg2a',
       'fg2_pct', 'fg3a_frequency', 'fg3m', 'fg3a', 'fg3_pct'],
      dtype='object')

In [298]:
#Get Player location stats
player_shot_location = json.loads(leaguedashplayershotlocations.LeagueDashPlayerShotLocations(season_type_all_star = 'Regular Season', season = '2021-22', distance_range = 'By Zone').get_json())

In [299]:
#Get both level of labels and combine them for a single level
first_level_column_names = player_shot_location['resultSets']['headers'][0]['columnNames']
second_level_column_names = player_shot_location['resultSets']['headers'][1]['columnNames']

first_level_column_names = [s.replace(" ", "_").lower() for s in first_level_column_names]
second_level_column_names = [s.lower() for s in second_level_column_names]
repetitive_columns = ['fgm', 'fga', 'fg_pct']
converted_column_names = [x + "_" + y for x in first_level_column_names for y in repetitive_columns]

second_level_column_names[6:] = converted_column_names
player_shot_location_df = pd.DataFrame(player_shot_location['resultSets']['rowSet'], columns = second_level_column_names)
player_shot_location_df = player_shot_location_df.drop(columns = ['player_name',  
                                                      'team_id', 'team_abbreviation', 'age', 'nickname'], axis = 1)

In [300]:
player_shot_location_df

,player_id,restricted_area_fgm,restricted_area_fga,restricted_area_fg_pct,in_the_paint_(non-ra)_fgm,in_the_paint_(non-ra)_fga,in_the_paint_(non-ra)_fg_pct,mid-range_fgm,mid-range_fga,mid-range_fg_pct,...,right_corner_3_fg_pct,above_the_break_3_fgm,above_the_break_3_fga,above_the_break_3_fg_pct,backcourt_fgm,backcourt_fga,backcourt_fg_pct,corner_3_fgm,corner_3_fga,corner_3_fg_pct
0,203932,277.0,367.0,0.755,29,98,0.296,41.0,109.0,0.376,...,0.356,49.0,144.0,0.340,0.0,4.0,0.0,38.0,112.0,0.339
1,1630565,1.0,1.0,1.000,0,1,0.000,0.0,2.0,0.000,...,0.000,0.0,1.0,0.000,NaN,NaN,NaN,0.0,0.0,0.000
2,1628988,48.0,80.0,0.600,49,105,0.467,15.0,50.0,0.300,...,0.500,32.0,86.0,0.372,0.0,0.0,0.0,7.0,17.0,0.412
3,1630174,26.0,34.0,0.765,7,17,0.412,8.0,16.0,0.500,...,0.211,22.0,71.0,0.310,0.0,0.0,0.0,9.0,44.0,0.205
4,1630598,71.0,110.0,0.645,39,83,0.470,4.0,7.0,0.571,...,0.261,25.0,84.0,0.298,0.0,0.0,0.0,17.0,53.0,0.321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,203897,240.0,349.0,0.688,39,115,0.339,101.0,247.0,0.409,...,0.583,147.0,402.0,0.366,0.0,0.0,0.0,38.0,73.0,0.521
601,1630285,9.0,17.0,0.529,9,22,0.409,0.0,5.0,0.000,...,0.000,1.0,8.0,0.125,0.0,0.0,0.0,0.0,0.0,0.000
602,1630192,51.0,76.0,0.671,4,18,0.222,4.0,12.0,0.333,...,0.529,20.0,46.0,0.435,0.0,0.0,0.0,17.0,34.0,0.500
603,1630533,70.0,84.0,0.833,21,51,0.412,24.0,49.0,0.490,...,0.447,35.0,118.0,0.297,0.0,2.0,0.0,41.0,120.0,0.342


In [301]:
test2 = pd.merge(test, player_shot_location_df, left_on = 'PLAYER_ID', right_on = 'player_id', how = 'outer')

In [302]:
test2

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,right_corner_3_fg_pct,above_the_break_3_fgm,above_the_break_3_fga,above_the_break_3_fg_pct,backcourt_fgm,backcourt_fga,backcourt_fg_pct,corner_3_fgm,corner_3_fga,corner_3_fg_pct
0,203932,Aaron Gordon,Aaron,1610612743,DEN,26.0,75,46,29,0.613,...,0.356,49.0,144.0,0.340,0.0,4.0,0.0,38.0,112.0,0.339
1,1630565,Aaron Henry,Aaron,1610612755,PHI,22.0,6,6,0,1.000,...,0.000,0.0,1.0,0.000,NaN,NaN,NaN,0.0,0.0,0.000
2,1628988,Aaron Holiday,Aaron,1610612756,PHX,25.0,63,34,29,0.540,...,0.500,32.0,86.0,0.372,0.0,0.0,0.0,7.0,17.0,0.412
3,1630174,Aaron Nesmith,Aaron,1610612738,BOS,22.0,52,32,20,0.615,...,0.211,22.0,71.0,0.310,0.0,0.0,0.0,9.0,44.0,0.205
4,1630598,Aaron Wiggins,Aaron,1610612760,OKC,23.0,50,13,37,0.260,...,0.261,25.0,84.0,0.298,0.0,0.0,0.0,17.0,53.0,0.321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,203897,Zach LaVine,Zach,1610612741,CHI,27.0,67,37,30,0.552,...,0.583,147.0,402.0,0.366,0.0,0.0,0.0,38.0,73.0,0.521
601,1630285,Zavier Simpson,Zavier,1610612760,OKC,25.0,4,1,3,0.250,...,0.000,1.0,8.0,0.125,0.0,0.0,0.0,0.0,0.0,0.000
602,1630192,Zeke Nnaji,Zeke,1610612743,DEN,21.0,41,24,17,0.585,...,0.529,20.0,46.0,0.435,0.0,0.0,0.0,17.0,34.0,0.500
603,1630533,Ziaire Williams,Ziaire,1610612763,MEM,20.0,62,41,21,0.661,...,0.447,35.0,118.0,0.297,0.0,2.0,0.0,41.0,120.0,0.342


In [303]:
#Get a DataFrame of all active NBA players and their ID's
nba_players = pd.json_normalize(players.get_active_players())

#Initialize a DataFrame to store all players career stats
career_stats_by_season = pd.DataFrame()

#Get a DataFrame of all teams and their ID's
nba_teams = pd.json_normalize(teams.get_teams())

#Initialize a DataFrame to store all teams season stats
team_stats_by_season = pd.DataFrame()

#Initialize a DataFrame for player shot % stats
player_shot_stats_df = pd.DataFrame()


In [304]:
#Pulling NBA players historical data
seasons = ['2021-22', '2020-21','2019-20','2018-19','2017-18','2016-17','2015-16','2014-15','2013-14','2012-13',
           '2011-12','2010-11','2009-10','2008-09','2007-08','2006-07','2005-06','2004-05','2003-04','2002-03','2001-02',
          '2000-01','1999-00','1998-99','1997-98','1996-97']
for s_id in seasons:
    #Player advanced stats
    player_season_stats = leaguedashplayerstats.LeagueDashPlayerStats(measure_type_detailed_defense = 'Advanced', 
                                        season_type_all_star = 'Regular Season', season = s_id).get_data_frames()[0]
    player_season_stats['SEASON'] = s_id
    
    #Player shot % stats
    player_shot_stats = leaguedashplayerptshot.LeagueDashPlayerPtShot( 
                                        season_type_all_star = 'Regular Season', season = s_id).get_data_frames()[0]
    #Manipulating player shot % data
    player_shot_stats.columns = player_shot_stats.columns.str.lower()
    player_shot_stats = player_shot_stats.drop(columns = ['player_last_team_id', 'player_name', 'player_last_team_abbreviation', 'gp', 'g', 'fga_frequency', 'fgm', 'fga', 'fg_pct'], axis = 1)
    
    #merge the first two datasets
    advanced_shots_merged = pd.merge(player_season_stats, player_shot_stats, left_on = 'PLAYER_ID', right_on = 'player_id', how = 'left')


    #Get Player location stats
    player_shot_location = json.loads(leaguedashplayershotlocations.LeagueDashPlayerShotLocations(season_type_all_star = 'Regular Season', season = s_id, distance_range = 'By Zone').get_json())
    #Get both level of labels and combine them for a single level
    first_level_column_names = player_shot_location['resultSets']['headers'][0]['columnNames']
    second_level_column_names = player_shot_location['resultSets']['headers'][1]['columnNames']

    first_level_column_names = [s.replace(" ", "_").lower() for s in first_level_column_names]
    second_level_column_names = [s.lower() for s in second_level_column_names]
    repetitive_columns = ['fgm', 'fga', 'fg_pct']
    converted_column_names = [x + "_" + y for x in first_level_column_names for y in repetitive_columns]

    second_level_column_names[6:] = converted_column_names
    player_shot_location_df = pd.DataFrame(player_shot_location['resultSets']['rowSet'], columns = second_level_column_names)
    player_shot_location_df = player_shot_location_df.drop(columns = ['player_name',  
                                                          'team_id', 'team_abbreviation', 'age', 'nickname'], axis = 1)    
        
    #merge the first merge with the last dataset
    all_data_merged = pd.merge(advanced_shots_merged, player_shot_location_df, left_on = 'PLAYER_ID', right_on = 'player_id', how = 'left')  
    
    time.sleep(2)
    
    career_stats_by_season = career_stats_by_season.append(all_data_merged)
    career_stats_by_season = career_stats_by_season.reset_index(drop = True)   
    print(s_id)

2021-22
2020-21
2019-20
2018-19
2017-18
2016-17
2015-16
2014-15
2013-14
2012-13
2011-12
2010-11
2009-10
2008-09
2007-08
2006-07
2005-06
2004-05
2003-04
2002-03
2001-02
2000-01
1999-00
1998-99
1997-98
1996-97


In [309]:
career_stats_by_season

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,right_corner_3_fg_pct,above_the_break_3_fgm,above_the_break_3_fga,above_the_break_3_fg_pct,backcourt_fgm,backcourt_fga,backcourt_fg_pct,corner_3_fgm,corner_3_fga,corner_3_fg_pct
0,203932,Aaron Gordon,Aaron,1610612743,DEN,26.0,75,46,29,0.613,...,0.356,49.0,144.0,0.340,0.0,4.0,0.0,38.0,112.0,0.339
1,1630565,Aaron Henry,Aaron,1610612755,PHI,22.0,6,6,0,1.000,...,0.000,0.0,1.0,0.000,NaN,NaN,NaN,0.0,0.0,0.000
2,1628988,Aaron Holiday,Aaron,1610612756,PHX,25.0,63,34,29,0.540,...,0.500,32.0,86.0,0.372,0.0,0.0,0.0,7.0,17.0,0.412
3,1630174,Aaron Nesmith,Aaron,1610612738,BOS,22.0,52,32,20,0.615,...,0.211,22.0,71.0,0.310,0.0,0.0,0.0,9.0,44.0,0.205
4,1630598,Aaron Wiggins,Aaron,1610612760,OKC,23.0,50,13,37,0.260,...,0.261,25.0,84.0,0.298,0.0,0.0,0.0,17.0,53.0,0.321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302,335,Willie Anderson,Willie,1610612748,MIA,30.0,28,22,6,0.786,...,0.750,1.0,3.0,0.333,0.0,0.0,0.0,5.0,7.0,0.714
12303,416,Willie Burton,Willie,1610612737,ATL,29.0,24,15,9,0.625,...,1.000,3.0,12.0,0.250,0.0,0.0,0.0,3.0,6.0,0.500
12304,1365,Xavier McDaniel,Xavier,1610612751,NJN,34.0,62,19,43,0.306,...,0.000,2.0,7.0,0.286,0.0,1.0,0.0,2.0,4.0,0.500
12305,212,Yinka Dare,Yinka,1610612751,NJN,24.0,41,12,29,0.293,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000


In [310]:
#career_stats_by_season.to_parquet('players_stats_by_season.parquet')  


In [125]:
#Pulling NBA teams historical data
seasons = ['2021-22', '2020-21','2019-20','2018-19','2017-18','2016-17','2015-16','2014-15','2013-14','2012-13',
           '2011-12','2010-11','2009-10','2008-09','2007-08','2006-07','2005-06','2004-05','2003-04','2002-03','2001-02',
          '2000-01','1999-00','1998-99','1997-98','1996-97']
for s_id in seasons:
    team_season_stats = leaguedashteamstats.LeagueDashTeamStats(measure_type_detailed_defense = 'Advanced', 
                                        season_type_all_star = 'Regular Season', season = s_id).get_data_frames()[0]
    team_season_stats['SEASON'] = s_id
    time.sleep(10)
    
    team_stats_by_season = team_stats_by_season.append(team_season_stats)
team_stats_by_season = team_stats_by_season.reset_index(drop = True)    

In [127]:
#Team Data Processing

#Save Teams Data as Parquet File
team_stats_by_season.to_parquet('team_stats_by_season.parquet')  


In [126]:
team_stats_by_season

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,...,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK,CFID,CFPARAMS,SEASON
0,1610612737,Atlanta Hawks,82,43,39,0.524,3941.0,114.0,115.4,112.1,...,12,14,1,8,6,17,14,10,Atlanta Hawks,2021-22
1,1610612738,Boston Celtics,82,51,31,0.622,3981.0,112.1,113.6,104.0,...,16,10,13,9,9,24,2,10,Boston Celtics,2021-22
2,1610612751,Brooklyn Nets,82,44,38,0.537,3951.0,111.0,113.2,109.6,...,30,15,18,11,11,11,11,10,Brooklyn Nets,2021-22
3,1610612766,Charlotte Hornets,82,43,39,0.524,3976.0,111.7,113.6,111.0,...,29,27,9,7,13,5,15,10,Charlotte Hornets,2021-22
4,1610612741,Chicago Bulls,82,46,36,0.561,3946.0,111.0,112.7,111.1,...,7,17,6,10,8,14,20,10,Chicago Bulls,2021-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,1610612760,Seattle SuperSonics,82,57,25,0.695,3956.0,108.2,109.6,99.5,...,21,15,6,6,4,16,3,10,Seattle SuperSonics,1996-97
768,1610612761,Toronto Raptors,82,30,52,0.366,3966.0,101.3,102.9,105.1,...,20,17,22,22,23,14,21,10,Toronto Raptors,1996-97
769,1610612762,Utah Jazz,82,64,18,0.780,3966.0,110.5,112.4,101.6,...,4,4,10,1,1,18,1,10,Utah Jazz,1996-97
770,1610612763,Vancouver Grizzlies,82,14,68,0.171,3956.0,97.6,99.1,108.9,...,29,29,20,26,28,24,29,10,Vancouver Grizzlies,1996-97


,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,...,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK,CFID,CFPARAMS
0,1610612737,Atlanta Hawks,82,43,39,0.524,3941.0,114.0,115.4,112.1,...,17,12,14,1,8,6,17,14,10,Atlanta Hawks
1,1610612738,Boston Celtics,82,51,31,0.622,3981.0,112.1,113.6,104.0,...,11,16,10,13,9,9,24,2,10,Boston Celtics
2,1610612751,Brooklyn Nets,82,44,38,0.537,3951.0,111.0,113.2,109.6,...,9,30,15,18,11,11,11,11,10,Brooklyn Nets
3,1610612766,Charlotte Hornets,82,43,39,0.524,3976.0,111.7,113.6,111.0,...,14,29,27,9,7,13,5,15,10,Charlotte Hornets
4,1610612741,Chicago Bulls,82,46,36,0.561,3946.0,111.0,112.7,111.1,...,28,7,17,6,10,8,14,20,10,Chicago Bulls
5,1610612739,Cleveland Cavaliers,82,44,38,0.537,3946.0,109.4,111.0,107.0,...,12,18,12,27,14,15,25,7,10,Cleveland Cavaliers
6,1610612742,Dallas Mavericks,82,52,30,0.634,3951.0,110.5,112.5,106.8,...,23,10,16,7,13,14,30,12,10,Dallas Mavericks
7,1610612743,Denver Nuggets,82,48,34,0.585,3961.0,111.7,113.8,110.3,...,19,1,4,26,1,1,20,9,10,Denver Nuggets
8,1610612765,Detroit Pistons,82,23,59,0.280,3956.0,103.3,105.6,111.1,...,13,26,30,20,30,29,13,29,10,Detroit Pistons
9,1610612744,Golden State Warriors,82,53,29,0.646,3946.0,110.5,112.1,103.9,...,15,6,8,29,3,4,15,3,10,Golden State Warriors
